Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [3]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
train.head()

id                                        description  ratingCategory
0  1321  \nSometimes, when whisky is batched, a few lef...               1
1  3861  \nAn uncommon exclusive bottling of a 6 year o...               0
2   655  \nThis release is a port version of Amrut’s In...               1
3   555  \nThis 41 year old single cask was aged in a s...               1
4  1965  \nQuite herbal on the nose, with aromas of dri...               1

In [7]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

### Define Pipeline Components

In [8]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
pipe = Pipeline([('vect', vect), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [46]:
parameters = {
    'vect__max_df': (0.1, 1.0),
    'vect__min_df': (0.01, 2.5),
    'clf__max_depth':(1,5,10,100,1000,10000, 20000)
}

rand_search = RandomizedSearchCV(pipe,parameters, cv=10, n_jobs=8, verbose=-1)
rand_search.fit(train['description'], train['ratingCategory'])

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   21.9s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   39.8s finished


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                       

In [45]:
rand_search.best_score_

0.7051628870726372

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [31]:
# Predictions on test sample
pred = rand_search.predict(test['description'])

In [32]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [33]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [15]:
subNumber = 0

In [34]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [57]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000, algorithm='randomized', n_iter=10)

In [50]:
lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [58]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[0.1, 0.5, 1.0],
    'clf__max_depth':(1,5,10,100,1000,5000,10000)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=-1)
grid_search.fit(train['description'], train['ratingCategory'])

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  3.0min
C:\Users\rowai\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 16.9min
[Parallel(n_jobs=4)]: Done 315 out of 315 | elapsed: 19.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [60]:
grid_search.best_score_

0.7274287526971178

### Make a Submission File

In [61]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [62]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [63]:
# Make Sure the Category is an Integer
submission.head()

id  ratingCategory
0  3461               1
1  2604               1
2  3341               1
3  3764               1
4  2306               1

In [64]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [1]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [ ]:
X = get_word_vectors(train['description'])
X_test = get_word_vectors(test['description'])

In [ ]:
rfc.fit(X, train['ratingCategory'])

In [ ]:
rfc.score(X, train['ratingCategory'])

In [ ]:
rfc.predict(X_test)

In [ ]:
test['ratingCategory'] = rfc.predict(X_test)

In [ ]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

Sentiment analysis is the action of guaging the feelings/emotion, i.e. sentiment, of words towards a specific topic.  It is different from document classification in various ways.  For example, in sentiment analysis the order of words matters, unlike in document classification which primarily looks at the distribution of words, their frequency of occurrence, etc.

we can create labeled sentiment data in a variety of ways:
1. Machine learning: predicting the polarity of sentiments based on train/test datasets.
2. Lexicon learning: using predefined lists of words that are already associated with specific sentiments
3. Hybrin learning: using a combination of machine learning and lexicon learning.


common applications of sentiment analysis includes:
- product reviews
- social media posts
- sentences, comments
- opinion discussions
- audience understanding, etc.

due to the complexity of human language, and the variety of vocabulary and context associated with human expression, it is very difficult for one nlp model to work on different datasets, even if they're related to the same category or topic.  Because our lecture notebook was in a more generalized domain, the general word embeddings were pretty good at predicting the topic an article belongs to, or a statement and its predicted topic.  However due to the specificity and the different domain that whiskey presents to the model and embeddings, the same model may not be as successful at making predictions in that domain.

Neural networks can improve this further because they operate more similar to the human brain (thus "neural") which can better interpret more complex meaning, and not just look at a "bag of words" but evaluate the context of the words, what comes before, what comes after, the intended meaning difference, etc.